# Vectorization

As mentioned before, this project is not meant to be a state of the art, blazing fast implementation of a neutral network. There is one optimization that I decided to implement to make it at least decent, i.e. the vectorization of matrix operations.

Given an input $x \in \mathbb{R}^n$, where $n$ is the input dimension, we know that the output is given by passing the input through the network, namely

$$ 
\begin{align}
z^1 &= x, \\
a^l &= W^l \cdot z^{(l-1)} + b^l, ~~~ &l = 2, 3 , \dots, L\\
z^ l &= \sigma(a^l), ~~~&l = 2, 3 , \dots, L
\end{align}
$$

Where $W^l, b^l$ represent the weight and biases of level $l$ respectively, and $\sigma$ is the non-linear activation function.  

We are using stochastic gradient descent with mini-batch size of $k$, which means we are go through $k$ examples before updating weights and biases. Instead of looping though each example (which would require $k$ matrix multiplications) we create a matrix of dimensions $(k, n)$, where each column is a data point. In that way, we need only one matrix multiplication for each mini-batch. In this scenario the above equations have to be interpreted as multiplications between matrices rather than matrix-vector.

In our code vectorization is handled in a few places. First of all both the input data and the labels are handled as `np.array` rather than simple Python lists. Such arrays are transposed in order to be consistent with the literature (and the above equations). 

In [ ]:
train_data = np.array(train_data)
train_labels = np.array(train_labels)
self.data = train_data.T
self.target = train_labels.T

Same applies to the testing data and labels (if provided):

In [ ]:
if test_data is not None and test_labels is not None:
    self.test_data = np.array(test_data).T
    self.test_labels = np.array(test_labels).T
    self.testing_error = []

The feed forward computations can be computed via

In [ ]:
for w, b in zip(self.weights, self.biases):
    result = np.dot(w, result) + b
    self.outputs.append(result)
    result = self.activation(result)
    self.activations.append(result)

In this phase saving the outputs and the activations is not stricly necessary, but we will need them later when computing the coefficient for the back propagation.

Note that there is also the option to absorb the biases into the weight matrix by adding them as a column (and a row of ones in the example matrix). I have decided not to do that to keep the code clearer.

## Back propagation

### Calculating the deltas

In order to make our network learn, we are going to use the backpropagation algorithm, which means we need a way to calculate the derivative of the cost function with respect to weights and biases. 

Let's start by stating the cost function $C$, in our case the mean squared error:

$$ 
C = \frac{1}{2} \sum_j (y_j - a_j)^2
$$

Here the $y_j$ are the target values and $a_j$ the corresponding activations (i.e. the network predictions).

Since we are using the sigmoid function $1/(1 - exp(x))$  as activation function, we know that, for each $l = 2, \dots, L$, the gradient of $C$ is given by

$$\begin{align}
\frac{\partial C}{\partial b^l_j} & = \delta^l_j, \\
\frac{\partial C}{\partial w^l_{j k}} & = a^{l-1}_k \cdot \delta^l_j.
\end{align}
$$
And the deltas are calculated via

$$\begin{align}
\delta^L & = \nabla_a C \cdot \sigma '(z^L), \\
\delta^l & = ((w^{l+1})^T \delta^{l+1}) \odot \sigma'(z^l).
\end{align}
$$

Note that all the previous are matrix equalities, including the deltas. In particular  $\delta^{L}$ has dimensions $(f, k)$, where $f$ is the dimension of the output layer and $k$ is the mini-batch dimension. In a similar way $\delta^l$ has the same shape of the outputs (or activations) at level $l$, i.e. $(d, k)$, with $d$ being the dimension of level $l$.



### Updating the weights and biases

Once we have calculated the deltas for all levels, we need to update both weights and biases

When updating biases we need to sum over all the example (i.e. sum all over the columns)

$$ b^l = b^l -  \frac{\eta}{k} \sum_{x}(\delta^{x,L}),
$$

where $\eta$ is the learning rate, $k$ is the dimension of the mini-batch and $\delta^{x, L}$ is the column of the matrix $\delta^L$ corresponding to the input $x$. Basically we are subtracting from $b^l$ a (scaled) sum of the columns of $\delta^{x, L}$.


In [ ]:
self.biases = [b - (learning_rate/total)* (np.sum(d, axis=1)).resape(b.shape) \
               for b, d  in zip(self.biases, self.deltas)]

The `np.sum(d, axis=1)` performs colum-wise summation.

Note that `.reshape(b.shape)` is fundamental, and was the source of a very hard bug to find.  `numpy` was broadcasting the result of the summation in a unwanted way.

In a similar fashion we update the weights via 

$$ w^l = \frac{\eta}{k} \sum_{x}(\delta^{x,L}\cdot(a^{x, L-1})^T) $$

in this case we don't need to explicitly sum over the columns, since this is automatically done by the matrix dot product. In the code this is performed via

In [ ]:
self.weights =  [w - (eta/total) * np.dot(d, a.T) \ 
                 for w, d, a in zip(self.weights, self.deltas, self.activations)]

## Activation functions

The activation functions need to be vectorized too. In this case, it means that when applied to an `np.array` the function is applied element wise.

For the sigmoid function 

    1 / (1 + np.exp(-x))
    
this is obtained for free thanks to the fact that `np.exp` is already vectorized. 

In the case of the rectified linear unit, we needed to add the `@np.vectorize` decorator to obtain such an effect.

# Error handling

Even if this code won't be used to train large networks, I have decided to implement some feature that I would have put in a 'real world' scenario.

The `NeuralNetwork` object can be initialized by either a `list` containig the shape of the network, or a `str`, the path to a JSON file, that contains a pre-trained model ([like this one](models/mnist.json)).

In case a string is passed, then we need to check that the file exists _and_ there is enough memory for all the weights. I perform both checks at once via:

In [ ]:
if isinstance(shape_or_file, str):
    try:
        self.load(shape_or_file)
    except FileNotFoundError:
        print(self.FILENOTFOUNDERROR_MESSAGE)
        raise
    except MemoryError:
        print(self.MEMORYERROR_MESSAGE)
        raise

Note that the `load` method checks that the JSON file contains the necessary data:

In [ ]:
def load(self, file_location):
    with open(file_location, 'r') as fp:
        data = json.load(fp)
    try:
        self.shape = data["shape"]
        self.weights = [np.array(w) for w in data["weights"]]
        self.biases = [np.array(b) for b in data["biases"]]
    except KeyError as e:
        print("Load failed, the json file does not contain the required key ", e)
        raise

Similary if the network is initialized with a `list`, we init all the internal variables with `np.array` of the right dimensions (containing zeroes), to make sure there is enough memory to train the network.

In [ ]:
elif isinstance(shape_or_file, list):
            if len(shape_or_file) < 2:
                print(self.WRONGSHAPE_MESSAGE)
                raise ValueError

            try:
                self.shape = shape_or_file
                self.activation = activation
                self._init_weights()
                self._init_biases()
                self._init_activations()
                self._init_outputs()
                if dropout:
                    self._init_dropouts()
            except MemoryError:
                print(self.MEMORYERROR_MESSAGE)
                raise
